In [1]:
import numpy as np
import torch
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from utils.Plots import PlotPitch, PlotGIF
from models.CopulaHMM import CopulaHMM

In [2]:
data=pd.read_csv('data/hulls_df_matchday2_reduced.csv')
events=pd.read_csv(f"data/matchday2_events.csv")
home_xy=pd.read_csv(f"data/home_xy.csv")
away_xy=pd.read_csv(f"data/away_xy.csv")
goals_info=events[events["Subtype"].isin(["ON TARGET-GOAL","HEAD-ON TARGET-GOAL","WOODWORK-GOAL"])]
home_goals=goals_info[goals_info["Team"]=="Home"]
away_goals=goals_info[goals_info["Team"]=="Away"]
shots_info=events[events["Type"]=="SHOT"]
home_shot=shots_info[shots_info["Team"]=="Home"]
away_shot=shots_info[shots_info["Team"]=="Away"]
sequence_XY = torch.tensor(data[["HomeHull","AwayHull"]].values/100)
completedata = pd.read_csv('data/hulls_df_matchday2.csv')

## Preliminary analysis

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(8, 5))
sns.histplot(data["HomeHull"], kde=True, color='blue',alpha=0.4,ax=axs[0],bins=50)
axs[0].axvline(data["HomeHull"].mean(), color='navy', linestyle='--', linewidth=2.5)
axs[0].set_xlabel("Convex Hull Area", fontsize=8, fontweight='normal');
axs[0].set_ylabel("Count", fontsize=8, fontweight='normal');
axs[0].set_title("Distribution of Home team's Convex Hull Area", fontsize=10, fontweight='bold');
sns.histplot(data["AwayHull"], kde=True, color='red',alpha=0.4,ax=axs[1],bins=50)
axs[1].axvline(data["AwayHull"].mean(), color='darkred', linestyle='--', linewidth=2.5)
axs[1].set_xlabel("Convex Hull Area", fontsize=8, fontweight='normal');
axs[1].set_ylabel("Count", fontsize=8, fontweight='normal');
axs[1].set_title("Distribution of Away team's Convex Hull Area", fontsize=10, fontweight='bold');
plt.subplots_adjust(hspace=0.5)
#plt.savefig("plots/ConvexHull_Distribution_matchday2.png", dpi=350, bbox_inches='tight')

In [ ]:
custom_palette = ["black", "#ff7f0e"]
plt.figure(figsize=(8, 6))
sns.scatterplot(data=data, x="HomeHull", y="AwayHull", hue="Period", palette=custom_palette, s=5)
plt.xlabel("Home team's Convex Hull Area", fontsize=10, fontweight='normal');
plt.ylabel("Away team's Convex Hull Area", fontsize=10, fontweight='normal');
plt.title("Correlation between Convex Hull Areas by Period", fontsize=12, fontweight='bold');
#plt.savefig("plots/ConvexHulls_correlation_matchday2.png", dpi=350, bbox_inches='tight')

## Viterbi&Co.

The model with 4 states is the most interesting from an interpretability point of view

In [3]:
posterior=torch.load(f"parameters/CopulaHMM_matchday2_4states.pt")

In [4]:
model=CopulaHMM.from_posterior(posterior)
MLS=model.viterbi(sequence_XY)
data["State"]=MLS.numpy()

In [5]:
model.print_estimates()

--------------------------------------------------------------------
STATE 0:
>> Mean of the Convex Hull for home team : 1185.22 m^2
>> Std of the Convex Hull for home team  : 340.21 m^2
>> Mean of the Convex Hull for away team : 1145.19 m^2
>> Std of the Convex Hull for away team  : 313.33 m^2
--------------------------------------------------------------------
STATE 1:
>> Mean of the Convex Hull for home team : 477.59 m^2
>> Std of the Convex Hull for home team  : 158.85 m^2
>> Mean of the Convex Hull for away team : 886.10 m^2
>> Std of the Convex Hull for away team  : 281.71 m^2
--------------------------------------------------------------------
STATE 2:
>> Mean of the Convex Hull for home team : 1013.96 m^2
>> Std of the Convex Hull for home team  : 274.37 m^2
>> Mean of the Convex Hull for away team : 649.33 m^2
>> Std of the Convex Hull for away team  : 196.21 m^2
--------------------------------------------------------------------
STATE 3:
>> Mean of the Convex Hull for home t

Let's now inspect the % of each state during the match:

In [ ]:
state_colors= {0:"#FD8033",
               1:"#0DC2B7",
               2:"#DAC11E",
               3:"#D964DC"}
plt.figure(figsize=(8, 6))
sns.countplot(data=data, x="State",stat="proportion",hue="State",palette=state_colors,edgecolor='black')
plt.xlabel("");
plt.xticks([0,1,2,3],["State 0","State 1","State 2","State 3"], fontsize=12)
plt.ylabel("Proportion", fontsize=12, fontweight='normal');
plt.yticks(fontsize=10)
plt.title("Proportion of each state during the match", fontsize=12, fontweight='bold');
plt.legend([],[], frameon=False);
#plt.savefig("plots/State_Proportion_matchday2.png", dpi=350, bbox_inches='tight')

## Different EPS assiciated to different states

Example of Home EPS > Away EPS

In [ ]:
PlotPitch(frame=51,
          lag=51,
          home_xy=home_xy,
          away_xy=away_xy,
          plotHulls=True,
          plotAllPlayers=True,
          title="")

In [ ]:
PlotPitch(frame=47701,
          lag=51,
          home_xy=home_xy,
          away_xy=away_xy,
          plotHulls=True,
          plotAllPlayers=True,
          title="")

Example of Home EPS < Away EPS

In [ ]:
PlotPitch(frame=30330,
          lag=51,
          home_xy=home_xy,
          away_xy=away_xy,
          plotHulls=True,
          plotAllPlayers=True,
          title="")

Example of Home EPS and Away EPS both small

In [ ]:
PlotPitch(frame=91010,
          lag=51,
          home_xy=home_xy,
          away_xy=away_xy,
          plotHulls=True,
          plotAllPlayers=True,
          title="")

Example of Home EPS and Away EPS both extended

In [ ]:
PlotPitch(frame=11720,
          lag=51,
          home_xy=home_xy,
          away_xy=away_xy,
          plotHulls=True,
          plotAllPlayers=True,
          title="")

# GIF

Kick off (start of the match)

In [ ]:
PlotGIF(home_xy, away_xy, initial_frame=51,final_frame=350,lag=51,step=2,
        gifname="gifs/plot_goal1_HomeTeam.gif",title="Kick off (start of the match)")

Goals:

In [ ]:
goals_info

NB: since the table above shows the starting frame of the goals, in the following GIFs we'll show the action from 3 seconds before (3 seconds= 75 frames before)

In [ ]:
PlotGIF(home_xy, away_xy, initial_frame=12202-75,final_frame=12212,lag=51,step=2,
        gifname="gifs/plot_goal1_HomeTeam.gif",title="Action leading to 1-0")

In [ ]:
PlotGIF(home_xy, away_xy, initial_frame=53049-75,final_frame=53075,lag=51,step=2,
        gifname="gifs/plot_goal1_AwayTeam.gif",title="Action leading to 1-1")

In [ ]:
PlotGIF(home_xy, away_xy, initial_frame=73983-75,final_frame=73995,lag=51,step=2,
        gifname="gifs/plot_goal2_HomeTeam.gif",title="Action leading to 2-1")

In [ ]:
PlotGIF(home_xy, away_xy, initial_frame=115009,final_frame=115024,lag=51, step=1,
        gifname="gifs/plot_goal2_AwayTeam.gif",title="Action leading to 2-2") #la palla è un po' sfasata sul rigore

In [ ]:
PlotGIF(home_xy, away_xy, initial_frame=121027-75,final_frame=121055,lag=51,step=2,
        gifname="gifs/plot_goal3_HomeTeam.gif",title="Action leading to 3-2")